In [ ]:
import gradio as gr
import psycopg2
import json
import numpy as np
from annoy import AnnoyIndex
from sentence_transformers import SentenceTransformer

#local import 
from search import search


def search_html(query):
    results = search(query,model, db_config)
    # Format the results as HTML-formatted text
    html_output = ""
    for project_id, chunk, distance in results:
        project = next((p for p in projects if p["ids"] == [project_id]), None)
        html_output += f"<h3>Project ID: {project_id}</h3>"
        html_output += f"<h4>Project title: {project['title']}</h4>"
        html_output += f"<p><b>Project abstract:</b> {project['abstract']}</p>"
        html_output += f"<p><b>Relevant snippet:</b> {chunk}</p>"
        html_output += f"<p><b>Distance:</b> {distance}</p>"
        html_output += "<hr>"
    return html_output


number_of_results = 5

# Database configuration
db_config = {
    'dbname': 'wb_s2_embeddings',
    'user': 's2',
    'password': 'wb@s2',
    'host': 'localhost',
    'port': 5432
}

model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

# Load the projects
with open("digital_agriculture_projects.json", "r") as f:
    projects = json.load(f)

# Define Gradio input and output interfaces
input_interface = gr.inputs.Textbox(lines=3, label="Enter your query")
output_interface = gr.outputs.HTML(label="Search Results")

# Create the Gradio interface
iface = gr.Interface(
    fn=search_html,
    inputs=input_interface,
    outputs=output_interface,
    title="Project Search",
    description="Search for relevant projects using natural language queries.",
)

# Launch the Gradio interface
iface.launch()
